# LIT 2023: Datenanalyse mit Python

Gert-Ludwig Ingold




<div style="margin-top: 7em;font-size: smaller;">Quellen: github.com/gertingold/lit2023</div>

### Tabellenkalkulation

* in der Windowswelt: Microsoft™ Excel™
* in der Linuxwelt (und darüber hinaus): LibreOffice
* Macros in LibreOffice können in Basic und Python programmiert werden

Alternative:
* Datenanalyse unabhängig von einem Tabellenkalkulationsprogramm
* Python mit der pandas-Bibliothek
* zusätzlich Jupyter-Notebook zur Entwicklung des Workflows und/oder zur Dokumentation der Datenanalyse

### Das erste Bild eines schwarzen Loches
<table>
 <tr style="background-color:#ffffff;">
 <td>
 <img src="eso1907a.jpg" width="500">
 <small>Credit: EHT Collaboration</small>
 </td><td>
 <p>verwendete Software:</p>
 <p>DiFX, CALC, PolConvert, HOPS, CASA, AIPS, ParselTongue, GNU Parallel, GILDAS, eht-imaging, <b>Numpy</b>, Scipy, <b>Pandas</b>, Astropy, <b>Jupyter</b>, <b>Matplotlib</b></p>
 <small>EHT Collaboration, Astrophys. J. Lett. <b>875</b>, L3 (2019)</small>
 </td>
 </tr>
</table>

### Pandas

Dokumentation: https://pandas.pydata.org/

Möglichkeiten zur Installation:
* Debian/Ubuntu: Paket python3-pandas installieren
* Anaconda-Distribution enthält pandas → https://anaconda.org
* in einem virtual environment: `pip install pandas`

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Passagierzählung S-Bahn Hamburg
### aus dem Open-Data-Portal der Deutschen Bahn AG

Webseite zum Datensatz:<br>
https://data.deutschebahn.com/dataset/passagierzahlung-s-bahn-hamburg.html

Daten zu Ein- und Aussteigern an den Hamburger S-Bahn Stationen aus Zählfahrten. (Rohdaten aus den Meßfahrten - exemplarische Werte, keine Gesamtpassagierzahlen)


Link zum Datensatz:<br>
https://download-data.deutschebahn.com/static/datasets/personenzahlen/Passagierzahlen.csv

In [ ]:
!head Passagierzahlen.csv

* Spalten sind durch Strichpunkte getrennt
* Encoding ist nicht UTF-8, sondern ISO8859
* Ein- und Aussteigerzahlen sind als Gleitkommazahlen angegeben, nicht als ganze Zahlen
* Es wird ein Dezimalkomma statt einem Dezimalpunkt verwendet
* Die Angabe des Datums erfolgt als String und erfordert für die weitere Verarbeitung eine Umwandlung in ein `datetime`-Objekt
* DS100 kurz: [Betriebsstellenverzeichnis](https://de.wikipedia.org/wiki/Betriebsstellenverzeichnis) (Druckschrift 100), zur Vervollständigung muss ein führendes A für H**A**mburg eingefügt werden

### Einlesen der Daten

In [ ]:
df = pd.read_csv('./Passagierzahlen.csv',
                 sep=';',
                 decimal=',',
                 encoding='ISO8859',
                 dtype={'Einsteiger': np.int64, 'Aussteiger': np.int64},
                 parse_dates=['dtmIstAnkunftDatum', 'dtmIstAbfahrtDatum'],
                 date_format='%d.%m.%Y %H:%M:%S'
                )

In [ ]:
df

In [ ]:
df['Station']

In [ ]:
df.Station

In [ ]:
type(df)

In [ ]:
type(df['DS100 kurz'])

* `DataFrame`: zweidimensionale Tabelle
* `Series`: Tabellenspalte

In [ ]:
df.info()

In [ ]:
df.keys()

In [ ]:
df.keys()[0]

### Gehen Passagiere verloren?

In [ ]:
df['Einsteiger'].sum()

In [ ]:
df['Aussteiger'].sum()

In [ ]:
(df.loc[:19, 'Einsteiger'] - df.loc[:19, 'Aussteiger']).cumsum()

### Hinzufügen einer Spalte mit den vollständigen Betriebsstellencodes

In [ ]:
df["DS100"] = "A" + df["DS100 kurz"]
df

## Haltestellendaten
### aus dem Open-Data-Portal der Deutschen Bahn AG

Webseite zum Datensatz:<br>
https://data.deutschebahn.com/dataset/data-haltestellen.html

Übersicht Haltestellen DB Station&Service AG


Link zum Datensatz:<br>
https://download-data.deutschebahn.com/static/datasets/haltestellen/D_Bahnhof_2017_09.csv

In [ ]:
!head D_Bahnhof_2017_09.csv

In [ ]:
!grep Schwabmünchen D_Bahnhof_2017_09.csv

* Encoding scheint UTF-8, also der Python-Standard, zu sein.

In [ ]:
stationen = pd.read_csv('./D_Bahnhof_2017_09.csv',
                        sep=';',
                        decimal=',',  
                        usecols=['DS100', 'NAME', 'LAENGE', 'BREITE'],
                        dtype={'LAENGE': np.float64, 'BREITE': np.float64}
                        )
stationen

* Es gibt Haltestelleneinträge ohne DS100-Code.
* NaN = Not a Number

### Zusammenführen der S-Bahn-Daten und der Haltestellendaten

* Haltestellendaten werden mit Hilfe der Spalte `DS100` zu den S-Bahn-Daten hinzugefügt.

In [ ]:
dfs = df.merge(stationen, how='left', on='DS100')

In [ ]:
dfs

* Es gibt Datensätze, bei denen die Haltestellendaten nicht hinzugefügt wurden, z.B. in den Datensätzen 0, 610667 und 610668.
* Was ist die Ursache?

In [ ]:
stationen[stationen['NAME'] == 'Aumühle']

* Es gibt Haltestellen mit zwei (oder womöglich noch mehr) DS100-Bezeichnern.
* Spalte Einträge dieser Art in mehrere Einträge mit nur einem DS100-Bezeichner auf.

* Insbesondere bei großen Datensätzen sollte man Schleifen über einzelne Zeilen vermeiden.
* Auch das Hinzufügen einzelner Datensätze kann aufwändig sein, da neue DataFrames erzeugt werden.
* Nach Möglichkeit Operationen verwenden, die auf den gesamten Datensatz wirken.

### Aufbereitung des Haltestellendatensatzes
1. Entferne Zeilen aus dem Datensatz, die in der `DS100`-Spalte den Eintrag `NaN` enthalten.
2. Zerlege die Zeichenketten in der `DS100`-Spalte in eine Liste mit ggf. mehreren DS100-Bezeichnern. Die Einträge sind durch Kommas getrennt.
3. Schreibe das Ergebnis wieder in die `DS100`-Spalte.
4. Lasse Zeilen mit mehreren DS100-Bezeichnern explodieren, d.h. erzeuge mehrere Zeilen mit jeweils nur einem DS100-Bezeichner.

In [ ]:
stationen_ohne_nan = stationen.dropna(subset='DS100')
spalte_ds100 = stationen_ohne_nan['DS100']
stationen['DS100'] = spalte_ds100.map(lambda x: x.split(','))

In [ ]:
stationen[stationen['NAME'] == 'Hamburg Berliner Tor']

In [ ]:
stationen = stationen.explode('DS100')
stationen[stationen['NAME'] == 'Hamburg Berliner Tor']

In [ ]:
dfs = df.merge(stationen, how='left', on='DS100')

In [ ]:
dfs


### Welche S-Bahnen gibt es in Hamburg?

In [ ]:
dfs['strKurzbezeichnung'].unique()

### Linienführung der S21

In [ ]:
dfs.loc[:4, ['Zugnr', 'Station', 'strKurzbezeichnung']]

In [ ]:
dfs.loc[17:22, ['Zugnr', 'Station', 'strKurzbezeichnung']]

In [ ]:
dfs.query("Zugnr == '248206'")

In [ ]:
datum = datetime.date(2016, 12, 10)
df_s21 = dfs.query("Zugnr == '248206' and dtmIstAnkunftDatum.dt.date == @datum")
df_s21

In [ ]:
plt.plot(df_s21['LAENGE'], df_s21['BREITE'], '-o')

### Was ist in Zeile 4 passiert?
* Indexzählung beginnt mit 0, betrachte also Index 3

In [ ]:
df_s21.iloc[3]

Bergedorf ist nicht Beringstedt!

In [ ]:
stationen.query("NAME == 'Hamburg-Bergedorf'")

In [ ]:
stationen.query("NAME == 'Beringstedt'")

### Korrigiere DS100-Code von Bergedorf nach `AGB S`
* Erzeuge eine Maske, die angibt, ob die Zeile den falschen DS100-Code enthält.
* Setze den Code in den betreffenden Zeilen des ursprünglichen S-Bahn-Datensatzes korrekt.

In [ ]:
is_bergedorf = df['DS100'] == 'ABGS'

In [ ]:
is_bergedorf

In [ ]:
df.loc[is_bergedorf, 'DS100'] = 'ABG S'

In [ ]:
df.iloc[:5]

### Nochmaliges Zusammenfügen der Daten

In [ ]:
dfs = df.merge(stationen, how='left', on='DS100')

In [ ]:
datum = datetime.date(2016, 12, 10)
df_s21 = dfs.query("Zugnr == '248206' and dtmIstAnkunftDatum.dt.date == @datum")

### Linienführung S21

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df_s21['LAENGE'], df_s21['BREITE'], '-o')
for x, y, text in zip(df_s21['LAENGE'], df_s21['BREITE'], df_s21['Station']):
    ax.annotate(text, xy=(x, y))

### Kann eine S-Bahn vor der Ankunft abfahren?

In [ ]:
df['Haltedauer'] = (df['dtmIstAbfahrtDatum']-df['dtmIstAnkunftDatum']).dt.total_seconds()

In [ ]:
df['Haltedauer'].min(), df['Haltedauer'].max()

In [ ]:
df[df['Haltedauer'] < 0]

In [ ]:
df.loc[df['Haltedauer'] < 0, 'Haltedauer'] = np.nan

In [ ]:
df.loc[df['Haltedauer'].isnull()]

### Verteilung der Haltedauern

In [ ]:
df['Haltedauer'].value_counts()

In [ ]:
plt.plot(df['Haltedauer'].value_counts(), '.')
plt.xlim(0, 100);

### Hält die S-Bahn am Hauptbahnhof länger?

In [ ]:
dfs['Station'].unique()

In [ ]:
hbfdata = df[df['Station'] == 'Hauptbahnhof']

In [ ]:
hbfdata

In [ ]:
plt.plot(hbfdata['Haltedauer'].value_counts(), '.')
plt.xlim(0, 100);

### Einordnung der Haltedauer in Kategorien

In [ ]:
bins = (0, 15, 35, 55, 100, 200, 20000)
pd.cut(df['Haltedauer'], bins=bins)

In [ ]:
labels = ['ultrakurz', 'kurz', 'mittel', 'lang', 'sehr lang', 'ultralang']
df['catHaltedauer'] = pd.cut(df['Haltedauer'], bins=bins, labels=labels)

In [ ]:
df

In [ ]:
df['catHaltedauer']

In [ ]:
df.loc[0, 'catHaltedauer'] < df.loc[1, 'catHaltedauer']

In [ ]:
df.loc[610668, 'catHaltedauer'] < df.loc[610669, 'catHaltedauer']

### Zahl der Ein- und Aussteiger im Laufe des Tages

In [ ]:
df['Ankunft'] = df['dtmIstAnkunftDatum'].dt.hour*60 + df['dtmIstAnkunftDatum'].dt.minute

In [ ]:
station = "Aumühle"
fig, (ax0, ax1) = plt.subplots(1, 2)
df_station = df.query("Station == @station")
ax0.plot(df_station['Ankunft'], df_station['Einsteiger'], 'r.')
ax1.plot(df_station['Ankunft'], df_station['Aussteiger'], 'b.')

In [ ]:
station = "Hauptbahnhof"
fig, (ax0, ax1) = plt.subplots(1, 2)
df_station = df.query("Station == @station")
ax0.plot(df_station['Ankunft'], df_station['Einsteiger'], 'r.')
ax1.plot(df_station['Ankunft'], df_station['Aussteiger'], 'b.')